In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import os
import numpy as np
import random
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pylab
import tensorflow as tf
import tensorflow.contrib.slim as slim
import tensorflow.contrib.slim.nets
from tensorflow.contrib.layers.python.layers import utils

import resnet_v2 as resnet
# import cv2
pylab.rcParams['figure.figsize'] = (10.0, 8.0)


# Initialize Dataset

In [2]:
def get_data(base_dir,image_dir,ann_file):
    image_path = '{}/images/{}'.format(baseDir,image_dir)
    ann_path='{}/annotations/{}.json'.format(baseDir,ann_file)

    return image_path, ann_path
    
# define the path to the annotation file corresponding to the images you want to work with
baseDir='/Users/kyle/Repositories/coco'

trainData='person_keypoints_train2014'
valData='person_keypoints_val2014'
testData='image_info_test-dev2015'

imageTrainDir = 'train2014'
imageValDir = 'val2014'
imageTestDir = 'test2015'

train_img_path, train_ann_path = get_data(baseDir,imageTrainDir,trainData)
val_img_path, val_ann_path = get_data(baseDir,imageValDir,valData)
# initialize a coco object
print("Initializing COCO objects to extract training and validation datasets...\n")
train_coco = COCO(train_ann_path)
val_coco = COCO(val_ann_path)
# get all images containing the 'person' category
train_catIds = train_coco.getCatIds(catNms=['person'])
train_imgIds = train_coco.getImgIds(catIds=train_catIds)
val_catIds = val_coco.getCatIds(catNms=['person'])
val_imgIds = val_coco.getImgIds(catIds=val_catIds)
# Just for dealing with the images on my computer (not necessary when working with the whole dataset)
# if args.small_dataset:
train_catIds = train_catIds[0:30]
train_imgIds = train_imgIds[0:30]
val_catIds = val_catIds[0:30]
val_imgIds = val_imgIds[0:30]


Initializing COCO objects to extract training and validation datasets...

loading annotations into memory...
Done (t=7.79s)
creating index...
index created!
loading annotations into memory...
Done (t=4.11s)
creating index...
index created!


In [3]:
#######################################################
############### VARIOUS HYPER-PARAMETERS ##############
#######################################################

NUM_KEYPOINTS = 17
BATCH_SIZE = 5 #args.batch_size
L = 5.0 # keypoint effective radius
D = 224 # image height and width
d = 56 # evaluation height and width (for mask and keypoint masks)

MASK_THRESHOLD = 0.5 # threshold for on/off prediction (in mask and keypoint masks)
KP_THRESHOLD = 0.5 # threshold for on/off prediction (in mask and keypoint masks)
KP_DISTANCE_THRESHOLD = 5.0 # threshold for determining if a keypoint estimate is accurate
X_INIT = tf.contrib.layers.xavier_initializer_conv2d() # xavier initializer for head architecture
# learning_rate1 = args.learning_rate1
# learning_rate2 = args.learning_rate2

#######################################################
#### VISUALIZATION TOOLS - WEIGHTS AND ACTIVATIONS ####
#######################################################
def highestPrimeFactorization(n):    
    return [(i, n//i) for i in range(1, int(n**0.5) + 1) if n % i == 0][-1] 

def getFilterImage(filters):
    """
    Takes as input a filter bank of size (1, H, W, C, D)
    Returns: a tensor of size (1, sqrt(D)*H, sqrt(D)*H, C)
    (This only really works for the first layer of filtes with an image as input)
    """
    padded_filters = tf.pad(filters,tf.constant([[0,0],[1,0],[1,0],[0,0],[0,0]]),'CONSTANT')
    filter_list = tf.unstack(padded_filters,axis=4)
    H,W = highestPrimeFactorization(len(filter_list))
    weight_strips = [tf.concat(filter_list[8*i:8*(i+1)],axis=1) for i in range(W)]
    weight_image = tf.concat(weight_strips,axis=2)
    return weight_image

def getActivationImage(activations):
    """
    Tiles an activation map into a square grayscale image
    Takes as input an activation map of size (N, H, W, D)
    Returns: a tensor of size (N, sqrt(D)*H, sqrt(D)*H, 1)
    """
    padded_activations = tf.pad(activations,tf.constant([[0,0],[1,0],[1,0],[0,0]]),'CONSTANT')
    expanded_activations = tf.expand_dims(padded_activations,axis=3)
    activations_list = tf.unstack(expanded_activations,axis=4)
    H,W = highestPrimeFactorization(len(activations_list))
    activation_strips = [tf.concat(activations_list[H*i:H*(i+1)],axis=1) for i in range(W)]
    activation_image = tf.concat(activation_strips,axis=2)
    return activation_image
#######################################################
##### PRE-PROCESSING AND DATASET EXTRACTION TOOLS #####
#######################################################
def extract_annotations_train(filename, imgID, coco=train_coco):
    anns = coco.loadAnns(coco.getAnnIds(imgID,catIds=[1],iscrowd=None))
    ann = max([ann for ann in anns], key=lambda item:item['area']) # extract annotation for biggest instance
    bbox = np.array(np.floor(ann['bbox']),dtype=int)
    keypoints = np.reshape(ann['keypoints'],(-1,3))
    mask = coco.annToMask(ann)
    return filename, bbox, keypoints, mask

def extract_annotations_val(filename, imgID, coco=val_coco):
    anns = coco.loadAnns(coco.getAnnIds(imgID,catIds=[1],iscrowd=None))
    ann = max([ann for ann in anns], key=lambda item:item['area']) # extract annotation for biggest instance
    bbox = np.array(np.floor(ann['bbox']),dtype=int)
    keypoints = np.reshape(ann['keypoints'],(-1,3))
    mask = coco.annToMask(ann)
    return filename, bbox, keypoints, mask

def preprocess_image_tf(filename, bbox_tensor, keypoints_tensor, mask, D=D):
    """
    Returns:
    resized_image (N,D,D,3) - cropped, padded (if needed), scaled to square image of size D
    resized_mask (N,D,D,1) - cropped, padded (if needed), scaled to square mask of size D
    pts (N,2,17) - keypoint coordinates (i,j) scaled to match up with resized_image
    labels (N,1,17) - values corresponding to pts: {0: invalid, 1:occluded, 2:valid}
    """
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.cast(image_decoded, tf.float32)

    mask = tf.transpose([mask],[1,2,0])
    bbox_tensor = tf.to_float(bbox_tensor)
    keypoints_tensor = tf.to_float(keypoints_tensor)

    sideLength = tf.reduce_max(bbox_tensor[2:],axis=0)
    centerX = tf.floor(bbox_tensor[0] + tf.divide(bbox_tensor[2],tf.constant(2.0)))
    centerY = tf.floor(bbox_tensor[1] + tf.divide(bbox_tensor[3],tf.constant(2.0)))
    center = tf.stack([centerX,centerY])

    corner1 = tf.to_int32(tf.minimum(tf.maximum(tf.subtract(center, tf.divide(sideLength,tf.constant(2.0))),0),
                        tf.reverse(tf.to_float(tf.shape(image)[:2]),tf.constant([0]))))
    corner2 = tf.to_int32(tf.minimum(tf.maximum(tf.add(center, tf.divide(sideLength,tf.constant(2.0))),0),
                        tf.reverse(tf.to_float(tf.shape(image)[:2]),tf.constant([0]))))
    i_shape = tf.subtract(corner2,corner1)
    d_shape = tf.subtract(tf.to_int32(sideLength),i_shape)

    scale = tf.divide(tf.constant(D,tf.float32), sideLength)
    cropped_image = tf.image.crop_to_bounding_box(image,corner1[1],corner1[0],
                                                tf.subtract(corner2,corner1)[1],tf.subtract(corner2,corner1)[0])
    cropped_mask = tf.image.crop_to_bounding_box(mask,corner1[1],corner1[0],
                                                tf.subtract(corner2,corner1)[1],tf.subtract(corner2,corner1)[0])

    dX = tf.floor(tf.divide(d_shape,tf.constant(2)))
    dY = tf.ceil(tf.divide(d_shape,tf.constant(2)))

    pts, labels = tf.split(keypoints_tensor,[2,1],axis=1)
    pts = tf.subtract(pts,tf.to_float(corner1)) # shift keypoints
    pts = tf.add(pts,tf.to_float(dX)) # shift keypoints
    pts = tf.multiply(pts,scale) # scale keypoints

    # set invalid pts to 0
    inbounds = tf.less(pts,tf.constant(D,tf.float32))
    inbounds = tf.multiply(tf.to_int32(inbounds), tf.to_int32(tf.greater(pts,0)))
    pts = tf.multiply(pts,tf.to_float(inbounds))
    pts = tf.transpose(pts,[1,0])
    labels = tf.transpose(labels,[1,0])

    padded_image = tf.image.pad_to_bounding_box(cropped_image,tf.to_int32(dX[1]),tf.to_int32(dX[0]),
                                                tf.to_int32(sideLength),tf.to_int32(sideLength))
    padded_mask = tf.image.pad_to_bounding_box(cropped_mask,tf.to_int32(dX[1]),tf.to_int32(dX[0]),
                                                tf.to_int32(sideLength),tf.to_int32(sideLength))

    resized_image = tf.image.resize_images(padded_image,tf.constant([D,D]),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # resized_image = resized_image - VGG_MEAN
    resized_mask = tf.image.resize_images(padded_mask,tf.constant([D,D]),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return resized_image, resized_mask, pts, labels

def scaleDownMaskAndKeypoints(image, mask, pts, labels, d=d, D=D):
    mask = tf.image.resize_images(mask,tf.constant([d,d]),tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    pts = tf.multiply(pts,tf.constant(d/D))
    return image, mask, pts, labels

def generate_keypoint_masks(image, mask, keypoints, labels, d=d, D=D, L=L):
    X, Y = tf.meshgrid(tf.linspace(0.0,d,d),tf.linspace(0.0,d,d))
    X = tf.reshape(X,[d,d,1])
    Y = tf.reshape(Y,[d,d,1])
    X_stack = tf.tile(X,tf.constant([1,1,17],dtype=tf.int32))
    Y_stack = tf.tile(Y,tf.constant([1,1,17],dtype=tf.int32))

    pts = tf.reshape(keypoints,[1,2,17])
    ptsX, ptsY = tf.split(pts,[1,1],axis=1)
    d1 = tf.square(tf.subtract(X_stack,ptsX))
    d2 = tf.square(tf.subtract(Y_stack,ptsY))

    pt_masks = tf.multiply(tf.divide(tf.constant(1.0),tf.add(d1,d2)+L),L)

    return image, mask, pt_masks, pts, labels

def generate_one_hot_keypoint_masks(image, mask, keypoints, labels, d=d):
    pts = tf.reshape(keypoints,[1,2,17])
    indices = tf.to_int32(pts)
    kp_mask1 = tf.one_hot(depth=d,indices=indices[:,1,:],axis=0)
    kp_mask2 = tf.one_hot(depth=d,indices=indices[:,0,:],axis=1)
    kp_masks = tf.matmul(tf.transpose(kp_mask1,(2,0,1)),tf.transpose(kp_mask2,(2,0,1)))
    kp_masks = tf.transpose(kp_masks,(1,2,0))
    return image, mask, kp_masks, pts, labels


#######################################################
################## SUMMARY DICTIONARY #################
#######################################################

image_summary_list = []
scalar_summary_list = []

#######################################################
################### PREPARE DATASET ###################
#######################################################
print("Initializing Dataset...\n")
with tf.variable_scope('DataSet'):
    ################### TRAIN DATASET ###################
    train_filenames = tf.constant(['{}/COCO_train2014_{:0>12}.jpg'.format(train_img_path, imgID) for imgID in train_imgIds])
    train_imgID_tensor = tf.constant(train_imgIds)
    train_dataset = tf.contrib.data.Dataset.from_tensor_slices((train_filenames, train_imgID_tensor))
    train_dataset = train_dataset.map(
        lambda filename, imgID: tf.py_func(extract_annotations_train, [filename, imgID], [filename.dtype, tf.int64, tf.int64, tf.uint8]))
    train_dataset = train_dataset.map(preprocess_image_tf)
    train_dataset = train_dataset.map(scaleDownMaskAndKeypoints)
    train_dataset = train_dataset.map(generate_one_hot_keypoint_masks)
    train_dataset = train_dataset.shuffle(buffer_size=10000)
    train_dataset = train_dataset.batch(BATCH_SIZE)

    #################### VAL DATASET ####################
    val_filenames = tf.constant(['{}/COCO_val2014_{:0>12}.jpg'.format(val_img_path, imgID) for imgID in val_imgIds])
    val_imgID_tensor = tf.constant(val_imgIds)
    val_dataset = tf.contrib.data.Dataset.from_tensor_slices((val_filenames, val_imgID_tensor))
    val_dataset = val_dataset.map(
        lambda filename, imgID: tf.py_func(extract_annotations_val,[filename, imgID],[filename.dtype, tf.int64, tf.int64, tf.uint8]))
    val_dataset = val_dataset.map(preprocess_image_tf)
    val_dataset = val_dataset.map(scaleDownMaskAndKeypoints)
    val_dataset = val_dataset.map(generate_one_hot_keypoint_masks)
    val_dataset = val_dataset.shuffle(buffer_size=10000)
    val_dataset = val_dataset.batch(BATCH_SIZE)

    iterator = tf.contrib.data.Iterator.from_structure(train_dataset.output_types, train_dataset.output_shapes)

    images, masks, kpt_masks, pts, labels = iterator.get_next()
    train_init_op = iterator.make_initializer(train_dataset)
    val_init_op = iterator.make_initializer(val_dataset)

    image_summary_list.append(tf.summary.image('keypoint_masks', getActivationImage(kpt_masks)))
    image_summary_list.append(tf.summary.image('input_images', images))

#######################################################
print("done")

Initializing Dataset...

done


In [4]:
def softmax_keypoint_masks(kpt_masks, d=56):
    return tf.reshape(tf.nn.softmax(tf.reshape(kpt_masks, [-1,d**2,17]),dim=1),[-1,d,d,17])

# ResNetMod

In [5]:
is_training = tf.placeholder(tf.bool)

resnet_v2 = tf.contrib.slim.nets.resnet_v2
with slim.arg_scope(resnet_v2.resnet_arg_scope()):
    logits, endpoints = resnet_v2.resnet_v2_50(
        inputs=images, num_classes=2, is_training=is_training, reuse=None,
        output_stride=16, scope='resnet_v2_50')

# Model Path to ResNet v2 50 checkpoint
model_path = '/Users/kyle/Documents/SCHOOL/Stanford/STANFORD_2016_to_2017/3rd_Quarter/CS231N/Project/checkpoints/resnet_v2_50.ckpt'
assert(os.path.isfile(model_path))
# Backbone Variables - remember to exclude all variables above backbone (including block4 and logits)
backbone_variables = tf.contrib.framework.get_variables_to_restore(exclude=['resnet_v2_50/postnorm','resnet_v2_50/logits'])
init_fn = tf.contrib.framework.assign_from_checkpoint_fn(model_path, backbone_variables) # Call to load pretrained weights


In [6]:
for k in endpoints.keys():
    print(k)

resnet_v2_50/conv1
resnet_v2_50/block1/unit_1/bottleneck_v2/shortcut
resnet_v2_50/block1/unit_1/bottleneck_v2/conv1
resnet_v2_50/block1/unit_1/bottleneck_v2/conv2
resnet_v2_50/block1/unit_1/bottleneck_v2/conv3
resnet_v2_50/block1/unit_1/bottleneck_v2
resnet_v2_50/block1/unit_2/bottleneck_v2/conv1
resnet_v2_50/block1/unit_2/bottleneck_v2/conv2
resnet_v2_50/block1/unit_2/bottleneck_v2/conv3
resnet_v2_50/block1/unit_2/bottleneck_v2
resnet_v2_50/block1/unit_3/bottleneck_v2/conv1
resnet_v2_50/block1/unit_3/bottleneck_v2/conv2
resnet_v2_50/block1/unit_3/bottleneck_v2/conv3
resnet_v2_50/block1/unit_3/bottleneck_v2
resnet_v2_50/block1
resnet_v2_50/block2/unit_1/bottleneck_v2/shortcut
resnet_v2_50/block2/unit_1/bottleneck_v2/conv1
resnet_v2_50/block2/unit_1/bottleneck_v2/conv2
resnet_v2_50/block2/unit_1/bottleneck_v2/conv3
resnet_v2_50/block2/unit_1/bottleneck_v2
resnet_v2_50/block2/unit_2/bottleneck_v2/conv1
resnet_v2_50/block2/unit_2/bottleneck_v2/conv2
resnet_v2_50/block2/unit_2/bottleneck_v

In [7]:
def bilinear_filter(channels_in,channels_out):
    f = tf.multiply(tf.constant([0.5, 1.0, 0.5],shape=[3,1]),tf.constant([0.5, 1.0, 0.5],shape=[1,3]))
    f = tf.stack([f for i in range(channels_out)],axis=2)
    f = tf.stack([f for i in range(channels_in)],axis=3)
    return f

In [8]:
block1 = endpoints['resnet_v2_50/block1']
block2 = endpoints['resnet_v2_50/block2']
block3 = endpoints['resnet_v2_50/block3']
block4 = endpoints['resnet_v2_50/block4']

In [9]:
block1.shape.as_list(),block2.shape.as_list(),block3.shape.as_list(),block4.shape.as_list()

([None, 28, 28, 256],
 [None, 14, 14, 512],
 [None, 14, 14, 1024],
 [None, 14, 14, 2048])

In [10]:
with tf.variable_scope('block3_head'):
    # initialize as bilinear upsampling
    input_shape=block3.shape.as_list()
    w3 = bilinear_filter(input_shape[3],128)
    b3 = tf.nn.conv2d_transpose(block3,w3,output_shape=tf.constant(
            [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
    b3 = tf.layers.batch_normalization(b3,axis=3) # axis=3 --> IMPORTANT!!
    b3 = tf.nn.relu(b3)
with tf.variable_scope('block4_head'):
    input_shape=block4.shape.as_list()
    w4 = bilinear_filter(input_shape[3],128)
    b4 = tf.nn.conv2d_transpose(block4,w4,output_shape=tf.constant(
            [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
    b4 = tf.layers.batch_normalization(b4,axis=3) # axis=3 --> IMPORTANT!!
    b4 = tf.nn.relu(b4)

In [11]:
b3.shape.as_list(), b4.shape.as_list()

([1, 28, 28, 128], [1, 28, 28, 128])

In [12]:
net = tf.concat([b3,b4],axis=3)

In [13]:
net.shape.as_list()

[1, 28, 28, 256]

In [16]:
input_shape=net.shape.as_list()
w = bilinear_filter(input_shape[3],128)
net = tf.nn.conv2d_transpose(net,w,output_shape=tf.constant(
        [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
net = tf.layers.batch_normalization(net,axis=3) # axis=3 --> IMPORTANT!!
net = tf.nn.relu(net)

In [17]:
net.shape.as_list()

[1, 56, 56, 128]

In [18]:
logits = tf.layers.conv2d(net,17,(1,1),(1,1),kernel_initializer=X_INIT)

In [24]:
logits.shape.as_list()

[1, 56, 56, 17]

In [20]:
with tf.variable_scope('keypoint_predictions'):
    keypoint_predictions = softmax_keypoint_masks(logits, d=56)

In [21]:
keypoint_predictions.shape.as_list()

[1, 56, 56, 17]

In [ ]:
block1 = endpoints['resnet_v2_50/block1']
block2 = endpoints['resnet_v2_50/block2']
block3 = endpoints['resnet_v2_50/block3']
block4 = endpoints['resnet_v2_50/block4']

with tf.variable_scope('block3_head'):
    # initialize as bilinear upsampling
    input_shape=block3.shape.as_list()
    w3 = bilinear_filter(input_shape[3],128)
    b3 = tf.nn.conv2d_transpose(block3,w3,output_shape=tf.constant(
            [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
    b3 = tf.layers.batch_normalization(b3,axis=3) # axis=3 --> IMPORTANT!!
    b3 = tf.nn.relu(b3)
with tf.variable_scope('block4_head'):
    input_shape=block4.shape.as_list()
    w4 = bilinear_filter(input_shape[3],128)
    b4 = tf.nn.conv2d_transpose(block4,w4,output_shape=tf.constant(
            [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
    b4 = tf.layers.batch_normalization(b4,axis=3) # axis=3 --> IMPORTANT!!
    b4 = tf.nn.relu(b4)
    
net = tf.concat([b3,b4],axis=3)
input_shape=net.shape.as_list()
w = bilinear_filter(input_shape[3],128)
net = tf.nn.conv2d_transpose(net,w,output_shape=tf.constant(
        [input_shape[0],2*input_shape[1],2*input_shape[2],128]),strides=(2,2),padding='SAME')
net = tf.layers.batch_normalization(net,axis=3) # axis=3 --> IMPORTANT!!
net = tf.nn.relu(net)

logits = tf.layers.conv2d(net,17,(1,1),(1,1),kernel_initializer=X_INIT)

with tf.variable_scope('keypoint_predictions'):
    keypoint_predictions = softmax_keypoint_masks(logits, d=56)

### Keypoint One-hot masks

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
sess.run(train_init_op)

In [ ]:
f = bilinear_filter(128,24)
f.shape.as_list()

In [ ]:
image = tf.placeholder(tf.float32,[1,20,20,128])

In [ ]:
net = tf.layers.conv2d_transpose(image,strides=(2,2),kernel_initializer=f)

In [ ]:
tf.nn.conv2d_transpose(image)

In [ ]:
def SoftmaxKeypointMask(kpt_masks, d=56):
    return tf.reshape(tf.nn.softmax(tf.reshape(kpt_masks, [-1,d**2,17]),dim=1),[-1,d,d,17])

In [ ]:
SoftmaxKeypointMask(kpt_masks, d=56)

In [ ]:
w = tf.Variable(bilinear_filter(128,24))

In [ ]:
w.shape

In [ ]:
image.shape

In [ ]:
tf.nn.conv2d_transpose(image,w,tf.constant([1,40,40,24]),strides=(2,2),padding='SAME')

In [23]:
53312/(56**2)

17.0

In [25]:
os.path.isdir('/tmp/KyleNet')

True